In [ ]:
#김도현_데이터수집_과제2(yes24책스크랩)
#책종류 검색 - 메이저리그
#1. 책표지 이미지 다운로드 받기
# 2. 책제목 저자 출판사 출판일 가격 평점 스크랩
# 3. 책제목을 클립 리뷰 스크랩
# 4. mysql 저장
# 책제목 - title
# 저자 - author
# 출판사 - publisher
# 출판일 - publication_date
# 가격 - price
# 평점 - rating
# title,author,publisher,publication_date,price,rating

In [1]:
import pymysql

def insert_book_info(data):
    conn = pymysql.connect(host='localhost',
            user='root',
            password='root123',
            db='pythondb',
            charset='utf8')
    cursor = conn.cursor()
    sql='insert into book_info(title,author,publisher,publication_date,price,rating) \
        values(%s, %s, %s, %s, %s, %s)'
    cursor.execute(sql, data)
    conn.commit()
    conn.close()
    
def select_book_id(data):
    conn = pymysql.connect(host='localhost',
            user='root',
            password='root123',
            db='pythondb',
            charset='utf8')
    cursor = conn.cursor()
    sql="select id from book_info where title=%s"
    cursor.execute(sql,data)
    result=cursor.fetchone()
    conn.commit()
    conn.close()
    return result[0]

def insert_book_review(data_total):
    conn = pymysql.connect(host='localhost',
            user='root',
            password='root123',
            db='pythondb',
            charset='utf8')
    cursor = conn.cursor()
    sql='insert into book_review(book_id,review) values(%s, %s)'
    cursor.executemany(sql, data_total)
    conn.commit()
    conn.close()

In [2]:
from urllib.request import urlretrieve
def image_save(src,title): # 이미지 저장 함수
    title=title.replace('/','')
    savename='/home/dohyun/pyphonwork/bookimage/'+title+'.jpg'
    urlretrieve(src,savename)
    print('saved')

In [3]:
def book_infos(): #책 정보 저장 함수
    temp1=driver.find_elements_by_css_selector('#yDetailTopWrap > div.topColRgt > div.gd_infoTop')
    title=temp1[0].find_element_by_css_selector('div > h2').text
#     print(title)
    image_link=driver.find_element_by_css_selector('#yDetailTopWrap > div.topColLft > div > span > em > img')
    src=image_link.get_attribute('src')
    image_save(src,title)
    # 저자 - author
    author_temp=temp1[0].find_elements_by_css_selector('span.gd_pubArea > span.gd_auth')
    if(len(author_temp)>0):
        author=author_temp[0].text
    else:
        author=''
#     print(author)
    # 출판사 - publisher
    publisher_temp=temp1[0].find_elements_by_css_selector('span.gd_pubArea > span.gd_pub')
    if(len(publisher_temp)>0):
        publisher=publisher_temp[0].text
    else:
        publisher=''
#         print(publisher)
    # 출판일 - publication_date
    publication_date_temp=temp1[0].find_elements_by_css_selector('span.gd_pubArea > span.gd_date')
    if(len(publication_date_temp)>0):
        publication_date=publication_date_temp[0].text
    else:
        publication_date=''
#     print(publication_date)
    # 가격 - price
    price=driver.find_element_by_css_selector('#yDetailTopWrap > div.topColRgt > div.gd_infoBot \
        > div.gd_infoTbArea > div > table > tbody > tr:nth-child(2) > td > span').text
#     print(price)
    # 평점 - rating
    rating=''
    rating_temp=driver.find_elements_by_css_selector('#spanGdRating > a > em')
    if(len(rating_temp)>0):
        rating=rating_temp[0].text
    else:
        rating=''
    
    bookinfo=[title,author,publisher,publication_date,price,rating]
    insert_book_info(bookinfo)
    bookinfo_list.append(bookinfo)
    

In [4]:
def review_save():
    try:
        while True :
            temp1=driver.find_elements_by_css_selector('#yDetailTopWrap > div.topColRgt > div.gd_infoTop')
            title=temp1[0].find_element_by_css_selector('div > h2').text
            book_id=select_book_id(title)
            readLinks = driver.find_elements_by_css_selector('#infoset_reviewContentList div.reviewInfoBot.crop \
                                                             > a > div > span')
            if(len(readLinks) == 0):
                return

            for readlink in readLinks :
                readlink.click()
                time.sleep(1)

            reviewList = driver.find_elements_by_css_selector('#infoset_reviewContentList > \
                                                              div.reviewInfoGrp.lnkExtend.infoMoreSubOn > div.reviewInfoBot.origin > div.review_cont')
    #         temp_list = []

            time.sleep(3)         
            for rev in reviewList :
                review=rev.text
                review_list.append([book_id,review])
    #         print(review_list)
            stopFlag = False
            
            for n in range(4, 13) :
                linkurl = '#infoset_reviewContentList > div.review_sort.sortBot > \
                div.review_sortLft > div > a:nth-child('+str(n)+')'
                linkNum = driver.find_element_by_css_selector(linkurl)
                print(linkNum.get_attribute('class'))
                
                if(linkNum.get_attribute('class') == "bgYUI next dim"):
                    stopFlag = True
                    break;
                    
                linkNum.click()
                time.sleep(3)
                
                readLinks = driver.find_elements_by_css_selector('#infoset_reviewContentList div.reviewInfoBot.crop > a')
            
                for readlink in readLinks :
                    readlink.click()
                    #driver.execute_script("arguments[0].click();", readlink)
                    time.sleep(3)

                reviewList = driver.find_elements_by_css_selector('#infoset_reviewContentList > \
                div.reviewInfoGrp.lnkExtend.infoMoreSubOn > div.reviewInfoBot.origin > div.review_cont')
                time.sleep(2)

                for rev in reviewList :
                    review=rev.text
                    review_list.append([book_id,review]) # 리뷰 temp_list에 저장
            
            if stopFlag == True :
                break
                
            nextPage = '#infoset_reviewContentList > div.review_sort.sortBot > \
            div.review_sortLft > div > a.bgYUI.next'
            linkNum = driver.find_element_by_css_selector(nextPage)
            linkNum.click()
            time.sleep(1)

    except Exception as ex:
        print("오류발생 : "+str(ex))

#     finally :
        

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome('/home/dohyun/pyphonwork/chromedriver')
driver.get('http://www.yes24.com')
driver.implicitly_wait(5)
search=driver.find_element_by_id('query')
search.send_keys('메이저리그')
search.send_keys(Keys.ENTER)
time.sleep(3) # 사이트 접속 - '조던피터슨' 검색 완료
bookinfo_list=[]
review_list=[]
for n in range(1, 40, 2) :
    bookurl='#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > \
    tbody > tr:nth-child('+str(n)+') > td.goods_infogrp > p.goods_name.goods_icon > a:nth-child(1)'
    booklink=driver.find_element_by_css_selector(bookurl)
    booklink.click()
    time.sleep(3)
    #title
    book_infos()
    review_save()

    driver.back()
    time.sleep(3)
    
driver.quit()
insert_book_review(review_list)

saved
num
bgYUI next dim
saved
saved
saved
bgYUI next dim
saved
saved
num
bgYUI next dim
saved
num
num
num
num
num
num
bgYUI next dim
saved
saved
saved
saved
saved
saved
saved
saved
saved
num
bgYUI next dim
saved
bgYUI next dim
saved
saved
saved
